In [3]:
import cv2
import mediapipe as mp
import pandas as pd
import joblib
import numpy as np
import time
import collections

# Load the trained model and label encoder
model = joblib.load("exercise_classifier_best.pkl")
le_label = joblib.load("label_encoder.pkl")

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Function to extract pose features with torso angle
def extract_pose_features(landmarks):
    if not landmarks or len(landmarks) < mp_pose.PoseLandmark.RIGHT_ANKLE.value + 1:
        return None

    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
               landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

    left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
    right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
    left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
    left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
    right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
    left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
    right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
    left_ankle_angle = calculate_angle(left_knee, left_ankle, [left_ankle[0] + 0.1, left_ankle[1]])
    right_ankle_angle = calculate_angle(right_knee, right_ankle, [right_ankle[0] + 0.1, right_ankle[1]])
    torso_angle = calculate_angle(left_hip, [(left_shoulder[0] + right_shoulder[0])/2, (left_shoulder[1] + right_shoulder[1])/2], left_knee)

    features = pd.DataFrame({
        'Shoulder_Angle': [left_shoulder_angle],
        'Elbow_Angle': [left_elbow_angle],
        'Hip_Angle': [left_hip_angle],
        'Knee_Angle': [left_knee_angle],
        'Ankle_Angle': [left_ankle_angle],
        'Shoulder_Ground_Angle': [right_shoulder_angle],
        'Elbow_Ground_Angle': [right_elbow_angle],
        'Hip_Ground_Angle': [right_hip_angle],
        'Knee_Ground_Angle': [right_knee_angle],
        'Ankle_Ground_Angle': [right_ankle_angle],
        'Torso_Angle': [torso_angle]
    })

    expected_order = ['Shoulder_Angle', 'Elbow_Angle', 'Hip_Angle', 'Knee_Angle',
                     'Ankle_Angle', 'Shoulder_Ground_Angle', 'Elbow_Ground_Angle',
                     'Hip_Ground_Angle', 'Knee_Ground_Angle', 'Ankle_Ground_Angle', 'Torso_Angle']
    features = features[expected_order]

    return features

# Function to classify the pose with adjustable confidence threshold
def classify_pose(features):
    if features is None or features.empty or features.shape[1] != 11:
        return "Unknown", 0.0
    try:
        # Drop Torso_Angle for prediction (assuming model wasn't trained with it)
        prediction_features = features.drop(columns=['Torso_Angle'])
        prediction = model.predict(prediction_features)[0]
        probability = model.predict_proba(prediction_features)[0]
        confidence = max(probability)
        # Print raw prediction for debugging
        print(f"Raw prediction: {le_label.inverse_transform([prediction])[0]}, Confidence: {confidence:.2f}")

        # Lowered threshold to 0.4 for imperfect postures
        if confidence < 0.4:
            # Fallback classification based on key angles
            elbow_angle = features['Elbow_Angle'].values[0]
            shoulder_angle = features['Shoulder_Angle'].values[0]
            if 70 < elbow_angle < 150:  # Typical elbow range for push-up
                return "push-up", 0.4  # Minimum confidence for fallback
            elif 90 < shoulder_angle < 180:  # Typical shoulder range for pull-up
                return "pull-up", 0.4
            return "Unknown", confidence
        exercise = le_label.inverse_transform([prediction])[0]
        return exercise, confidence
    except Exception as e:
        print(f"Classification error: {e}")
        return "Unknown", 0.0

# Function to count repetitions with posture tolerance
def count_repetitions(prev_exercise, current_exercise, prev_elbow_angle, current_elbow_angle):
    if prev_exercise and current_exercise in ["push-up", "pull-up"]:
        # Adjust threshold for imperfect postures (e.g., 20 degrees instead of 30)
        angle_change = abs(current_elbow_angle - prev_elbow_angle)
        if angle_change > 20:  # Looser threshold for rep counting
            return 1
    return 0

# Main function to process video or webcam with stabilized classification
def process_video(source=0):
    cap = cv2.VideoCapture(source) if isinstance(source, (int, str)) and source == 0 else cv2.VideoCapture(source)
    
    if not cap.isOpened():
        print(f"Error: Could not open video source {source}")
        return

    cv2.namedWindow('Exercise Classification', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Exercise Classification', 1280, 720)
    
    prev_exercise = "Unknown"
    prev_elbow_angle = 0
    rep_count = 0
    start_time = time.time()
    frame_count = 0
    exercise_queue = collections.deque(maxlen=5)  # Queue for smoothing over 5 frames

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        frame_count += 1
        elapsed_time = time.time() - start_time
        fps = frame_count / elapsed_time if elapsed_time > 0 else 0

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2)
            )

            features = extract_pose_features(results.pose_landmarks.landmark)
            if features is not None and not features.empty:
                exercise, confidence = classify_pose(features)
                exercise_queue.append(exercise)
                current_elbow_angle = features['Elbow_Angle'].values[0]

                # Stabilize exercise label by majority vote over the queue
                stabilized_exercise = max(set(exercise_queue), key=exercise_queue.count) if len(exercise_queue) == exercise_queue.maxlen else prev_exercise

                if stabilized_exercise in ["push-up", "pull-up"]:
                    rep_count += count_repetitions(prev_exercise, stabilized_exercise, prev_elbow_angle, current_elbow_angle)
                    prev_elbow_angle = current_elbow_angle
                prev_exercise = stabilized_exercise
            else:
                stabilized_exercise = "Unknown"
                confidence = 0.0

            cv2.putText(image, f'Exercise: {stabilized_exercise}', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f'Confidence: {confidence:.2f}', (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow('Exercise Classification', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage
if __name__ == "__main__":
    # Use 0 for webcam or provide video file path
    video_path = "video.mp4"  # Example video (update with correct path, e.g., "D:/Project/dataset/pull_up/pull-1.mp4")
    process_video(video_path)  # Change to 0 for webcam

Raw prediction: push-up, Confidence: 0.80
Raw prediction: push-up, Confidence: 0.85
Raw prediction: push-up, Confidence: 0.94
Raw prediction: push-up, Confidence: 0.95
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.97
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidenc

In [13]:
import cv2
import mediapipe as mp
import pandas as pd
import joblib
import numpy as np
import time
import collections

# Load the trained model and label encoder
model = joblib.load("exercise_classifier_best.pkl")
le_label = joblib.load("label_encoder.pkl")

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Function to extract pose features with torso angle
def extract_pose_features(landmarks):
    if not landmarks or len(landmarks) < mp_pose.PoseLandmark.RIGHT_ANKLE.value + 1:
        return None

    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
               landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

    left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
    right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
    left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
    left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
    right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
    left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
    right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
    left_ankle_angle = calculate_angle(left_knee, left_ankle, [left_ankle[0] + 0.1, left_ankle[1]])
    right_ankle_angle = calculate_angle(right_knee, right_ankle, [right_ankle[0] + 0.1, right_ankle[1]])
    torso_angle = calculate_angle(left_hip, [(left_shoulder[0] + right_shoulder[0])/2, (left_shoulder[1] + right_shoulder[1])/2], left_knee)

    features = pd.DataFrame({
        'Shoulder_Angle': [left_shoulder_angle],
        'Elbow_Angle': [left_elbow_angle],
        'Hip_Angle': [left_hip_angle],
        'Knee_Angle': [left_knee_angle],
        'Ankle_Angle': [left_ankle_angle],
        'Shoulder_Ground_Angle': [right_shoulder_angle],
        'Elbow_Ground_Angle': [right_elbow_angle],
        'Hip_Ground_Angle': [right_hip_angle],
        'Knee_Ground_Angle': [right_knee_angle],
        'Ankle_Ground_Angle': [right_ankle_angle],
        'Torso_Angle': [torso_angle]
    })

    expected_order = ['Shoulder_Angle', 'Elbow_Angle', 'Hip_Angle', 'Knee_Angle',
                     'Ankle_Angle', 'Shoulder_Ground_Angle', 'Elbow_Ground_Angle',
                     'Hip_Ground_Angle', 'Knee_Ground_Angle', 'Ankle_Ground_Angle', 'Torso_Angle']
    features = features[expected_order]

    return features

# Function to classify the pose with adjustable confidence threshold
def classify_pose(features):
    if features is None or features.empty or features.shape[1] != 11:
        return "Unknown", 0.0
    try:
        # Drop Torso_Angle for prediction (assuming model wasn't trained with it)
        prediction_features = features.drop(columns=['Torso_Angle'])
        prediction = model.predict(prediction_features)[0]
        probability = model.predict_proba(prediction_features)[0]
        confidence = max(probability)
        # Print raw prediction for debugging
        print(f"Raw prediction: {le_label.inverse_transform([prediction])[0]}, Confidence: {confidence:.2f}")

        # Lowered threshold to 0.4 for imperfect postures
        if confidence < 0.4:
            # Fallback classification based on key angles
            elbow_angle = features['Elbow_Angle'].values[0]
            shoulder_angle = features['Shoulder_Angle'].values[0]
            if 70 < elbow_angle < 150:  # Typical elbow range for push-up
                return "push-up", 0.4  # Minimum confidence for fallback
            elif 90 < shoulder_angle < 180:  # Typical shoulder range for pull-up
                return "pull-up", 0.4
            return "Unknown", confidence
        exercise = le_label.inverse_transform([prediction])[0]
        return exercise, confidence
    except Exception as e:
        print(f"Classification error: {e}")
        return "Unknown", 0.0

# Function to count repetitions with posture tolerance
def count_repetitions(prev_exercise, current_exercise, prev_elbow_angle, current_elbow_angle):
    if prev_exercise and current_exercise in ["push-up", "pull-up"]:
        # Adjust threshold for imperfect postures (e.g., 20 degrees instead of 30)
        angle_change = abs(current_elbow_angle - prev_elbow_angle)
        if angle_change > 20:  # Looser threshold for rep counting
            return 1
    return 0

# Main function to process video or webcam with stabilized classification
def process_video(source=0):
    cap = cv2.VideoCapture(source) if isinstance(source, (int, str)) and source == 0 else cv2.VideoCapture(source)
    
    if not cap.isOpened():
        print(f"Error: Could not open video source {source}")
        return

    cv2.namedWindow('Exercise Classification', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Exercise Classification', 1280, 720)
    
    prev_exercise = "Unknown"
    prev_elbow_angle = 0
    rep_count = 0
    start_time = time.time()
    frame_count = 0
    exercise_queue = collections.deque(maxlen=5)  # Queue for smoothing over 5 frames

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        frame_count += 1
        elapsed_time = time.time() - start_time
        fps = frame_count / elapsed_time if elapsed_time > 0 else 0

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2)
            )

            features = extract_pose_features(results.pose_landmarks.landmark)
            if features is not None and not features.empty:
                exercise, confidence = classify_pose(features)
                exercise_queue.append(exercise)
                current_elbow_angle = features['Elbow_Angle'].values[0]

                # Stabilize exercise label by majority vote over the queue
                stabilized_exercise = max(set(exercise_queue), key=exercise_queue.count) if len(exercise_queue) == exercise_queue.maxlen else prev_exercise

                if stabilized_exercise in ["push-up", "pull-up"]:
                    rep_count += count_repetitions(prev_exercise, stabilized_exercise, prev_elbow_angle, current_elbow_angle)
                    prev_elbow_angle = current_elbow_angle
                prev_exercise = stabilized_exercise
            else:
                stabilized_exercise = "Unknown"
                confidence = 0.0

            cv2.putText(image, f'Exercise: {stabilized_exercise}', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f'Confidence: {confidence:.2f}', (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow('Exercise Classification', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage
if __name__ == "__main__":
    # Use 0 for webcam or provide video file path
    video_path = "video.mp4"  # Example video (update with correct path, e.g., "D:/Project/dataset/pull_up/pull-1.mp4")
    process_video(video_path)  # Change to 0 for webcam

Raw prediction: push-up, Confidence: 0.80
Raw prediction: push-up, Confidence: 0.85
Raw prediction: push-up, Confidence: 0.94
Raw prediction: push-up, Confidence: 0.95
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.97
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidence: 0.98
Raw prediction: push-up, Confidenc

In [3]:
import matplotlib.pyplot as plt
import os
import numpy as np

# Create result folder if it doesn't exist
result_dir = "result"
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

# Define the metrics from the classification reports (based on your screenshot)
models = ['XGBoost', 'RandomForest']
classes = ['pull Up', 'push-up']

# Accuracy for both models (from your output: 0.975 78629434673)
accuracies = [0.97578629434673, 0.97578629434673]

# Precision, Recall, and F1-score for each class and model
# Values approximated from your screenshot and graph
precision = {
    'XGBoost': [0.98, 0.98],
    'RandomForest': [0.98, 0.98]
}
recall = {
    'XGBoost': [0.95, 0.99],
    'RandomForest': [0.95, 0.99]
}
f1_score = {
    'XGBoost': [0.96, 0.98],
    'RandomForest': [0.96, 0.98]
}

# Set up the plot
fig, ax = plt.subplots(figsize=(12, 6))

# Bar positions
bar_width = 0.2
index = np.arange(len(classes))
x_offsets = np.array([-1.5, -0.5, 0.5, 1.5]) * bar_width

# Plot bars for each metric
bars1 = ax.bar(index + x_offsets[0], [precision['XGBoost'][0], precision['XGBoost'][1]], bar_width, label='XGBoost Precision', color='skyblue')
bars2 = ax.bar(index + x_offsets[1], [recall['XGBoost'][0], recall['XGBoost'][1]], bar_width, label='XGBoost Recall', color='lightgreen')
bars3 = ax.bar(index + x_offsets[2], [f1_score['XGBoost'][0], f1_score['XGBoost'][1]], bar_width, label='XGBoost F1-Score', color='lightcoral')
bars4 = ax.bar(index + x_offsets[0], [precision['RandomForest'][0], precision['RandomForest'][1]], bar_width, label='RandomForest Precision', color='cyan', alpha=0.7)
bars5 = ax.bar(index + x_offsets[1], [recall['RandomForest'][0], recall['RandomForest'][1]], bar_width, label='RandomForest Recall', color='limegreen', alpha=0.7)
bars6 = ax.bar(index + x_offsets[2], [f1_score['RandomForest'][0], f1_score['RandomForest'][1]], bar_width, label='RandomForest F1-Score', color='salmon', alpha=0.7)

# Add accuracy as a horizontal line and text annotation
for i, acc in enumerate(accuracies):
    ax.axhline(y=acc, xmin=(i-0.4)/2, xmax=(i+0.4)/2, color='purple', linestyle='--', alpha=0.5)
    ax.text(i, acc + 0.005, f'Accuracy: {acc:.4f}', ha='center', va='bottom', color='purple', fontsize=10)

# Labels and titles
ax.set_xlabel('Classes')
ax.set_ylabel('Score')
ax.set_title('Model Performance Comparison')
ax.set_xticks(index)
ax.set_xticklabels(classes)
ax.set_ylim(0.9, 1.0)  # Focus on the relevant range
ax.legend(loc='lower center', bbox_to_anchor=(0.5, -0.2), ncol=3)

# Add value labels on bars
for bars in [bars1, bars2, bars3, bars4, bars5, bars6]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}', ha='center', va='bottom', fontsize=9)

# Adjust layout to prevent overlap
fig.tight_layout()

# Save the plot
plot_path = os.path.join(result_dir, "performance_comparison.png")
plt.savefig(plot_path, bbox_inches='tight')
plt.close()
print(f"Graph saved as {plot_path}")

Graph saved as result\performance_comparison.png
